All Imports

In [3]:
import os
import openpyxl
import requests
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup

Loading all the files

In [4]:
current_dir = os.getcwd()

In [5]:
def Loading_links():

        path = fr'{current_dir}\movielinks\review links'
        files = os.listdir(path)

        review_links = []              # Appending links stored in the file of each iteration

        for file in files:
            workbook = openpyxl.load_workbook(path+'/'+file)
            sheet = workbook['Sheet1']
            for row in sheet.iter_rows(min_row = 2 ,values_only=True):
                review_links.append(row[1])

        return review_links

review_links = Loading_links()

Scraping the reviews

In [6]:
save_path = fr'{current_dir}\Reviews'
timeout_seconds = 10            # Setting up time out to avoid connection loss

def scraper():

    os.makedirs(save_path, exist_ok=True)  

    with tqdm(total=len(review_links), desc="Processing", leave=True) as pbar:
        for link in review_links:

            reviews = [] 
            headers = {'User-Agent': 'MyIMDbScraper/1.0 (https://github.com/yourusername/my-imdb-scraper)'}
            
            try:
                page = requests.get(link, headers=headers, timeout=timeout_seconds).text
                soup = BeautifulSoup(page, 'html.parser')
                header = soup.find('div', class_='lister-item-header')
                title = header.find('a').text
                review = soup.find('div', class_='text show-more__control').text.strip()
                reviews.append(review)

                # Write or append the review to a file
                file_path = os.path.join(save_path, f'{title}.txt')
                if not os.path.exists(file_path):
                    with open(file_path, 'w', encoding='utf-8') as f:
                        f.write('\n'.join(reviews))
                else:
                    with open(file_path, 'a', encoding='utf-8') as f:
                        # Append a newline before adding the new review
                        f.write('\n\n')
                        f.write('\n'.join(reviews))

            except requests.Timeout:
                pass
            except Exception as e:
                pass

            pbar.update(1)  # Update the progress bar

scraper()


Processing: 100%|██████████| 19374/19374 [11:26:45<00:00,  2.13s/it]      
